In [1]:
! pip install -q transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [3]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [4]:
billsum = billsum.train_test_split(test_size=0.2)

In [5]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [6]:
example = billsum["train"][0]
for key in example:
    print("A key of the example: \"{}\"".format(key))
    print("\n\nThe value corresponding to the key-\"{}\"\n \"{}\"".format(key, example[key]))

A key of the example: "text"


The value corresponding to the key-"text"
 "The people of the State of California do enact as follows:


SECTION 1.
Section 8610.5 is added to the Government Code, to read:
8610.5.
(a) For purposes of this section:
(1) “Office” means the Office of Emergency Services.
(2) “Previous fiscal year” means the fiscal year immediately prior to the current fiscal year.
(3) “Utility” means an “electrical corporation” as defined in Section 218 of the Public Utilities Code.
(b) (1) State and local costs to carry out activities pursuant to this section and Chapter 4 (commencing with Section 114650) of Part 9 of Division 104 of the Health and Safety Code that are not reimbursed by federal funds shall be borne by a utility operating a nuclear powerplant with a generating capacity of 50 megawatts or more.
(2) The Public Utilities Commission shall develop and transmit to the office an equitable method of assessing a utility operating a powerplant for its reasonable share 

In [7]:
example =  billsum['train'][0]
pref_text = "summarize: " + example['text']

In [8]:
pref_text

'summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 8610.5 is added to the Government Code, to read:\n8610.5.\n(a) For purposes of this section:\n(1) “Office” means the Office of Emergency Services.\n(2) “Previous fiscal year” means the fiscal year immediately prior to the current fiscal year.\n(3) “Utility” means an “electrical corporation” as defined in Section 218 of the Public Utilities Code.\n(b) (1) State and local costs to carry out activities pursuant to this section and Chapter 4 (commencing with Section 114650) of Part 9 of Division 104 of the Health and Safety Code that are not reimbursed by federal funds shall be borne by a utility operating a nuclear powerplant with a generating capacity of 50 megawatts or more.\n(2) The Public Utilities Commission shall develop and transmit to the office an equitable method of assessing a utility operating a powerplant for its reasonable share of state agency costs specified in paragraph (1).\n(

## Preprocess
We will fine-tune the T5-small model. At the Overview page of the Hugging Face T5 model, it provides the following tips:

T5 is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks and for which each task is converted into a text-to-text format.
T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, e.g., for translation: translate English to German: …, for summarization: summarize: ….
We will load a T5 tokenizer to process text and summary and prepend a prefix "summarize: " for our text summarization task

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [10]:
tokenized_text = tokenizer(example['text'])
for key in tokenized_text:
    print(key)
    print(tokenized_text[key])

Token indices sequence length is longer than the specified maximum sequence length for this model (1641 > 512). Running this sequence through the model will result in indexing errors


input_ids
[37, 151, 13, 8, 1015, 13, 1826, 103, 3, 35, 2708, 38, 6963, 10, 180, 3073, 9562, 1300, 5568, 505, 4241, 12100, 19, 974, 12, 8, 3141, 3636, 6, 12, 608, 10, 505, 4241, 12100, 5, 41, 9, 61, 242, 3659, 13, 48, 1375, 10, 5637, 105, 22098, 153, 598, 8, 2126, 13, 15118, 1799, 5, 6499, 105, 10572, 19117, 5043, 215, 153, 598, 8, 5043, 215, 2017, 1884, 12, 8, 750, 5043, 215, 5, 10153, 105, 1265, 17, 14277, 153, 598, 46, 105, 17470, 138, 11861, 153, 38, 4802, 16, 5568, 204, 2606, 13, 8, 2575, 412, 17, 173, 2197, 3636, 5, 41, 115, 61, 5637, 1015, 11, 415, 1358, 12, 2331, 91, 1087, 19890, 288, 12, 48, 1375, 11, 8647, 314, 41, 287, 526, 4733, 28, 5568, 3, 18959, 15348, 61, 13, 2733, 668, 13, 6022, 3, 15442, 13, 8, 1685, 11, 6859, 3636, 24, 33, 59, 29560, 26, 57, 2822, 2731, 1522, 36, 3, 12940, 57, 3, 9, 6637, 2699, 3, 9, 6414, 579, 14925, 28, 3, 9, 3, 11600, 2614, 13, 943, 13950, 11876, 7, 42, 72, 5, 6499, 37, 2575, 412, 17, 173, 2197, 3527, 1522, 1344, 11, 9153, 12, 8, 828, 46, 30364, 15

We will create a function to preprocess the training and test data in batch. The preprocessing function will perform the following actions:

* Prepend the prefix "summarize: " to each text document to indicate to the T5 model that the task at hand is summarization.
* Convert the input texts and summary labels into a tokenized format that can be processed by the T5 model.
* Set the max_length parameter to ensure that the tokenized inputs and labels do not exceed a certain length, truncating any text that is too long.
* |Assign the tokenized labels to the labels field of model_inputs, which will be used during training to calculate the loss and optimize the model's parameters.

In [11]:
def preprocess_function(examples):
    # Prepends the string "summarize: " to each document in the 'text' field of the input examples.
    # This is done to instruct the T5 model on the task it needs to perform, which in this case is summarization.
    inputs = ["summarize: " + doc for doc in examples["text"]]

    # Tokenizes the prepended input texts to convert them into a format that can be fed into the T5 model.
    # Sets a maximum token length of 1024, and truncates any text longer than this limit.
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenizes the 'summary' field of the input examples to prepare the target labels for the summarization task.
    # Sets a maximum token length of 128, and truncates any text longer than this limit.
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    # Assigns the tokenized labels to the 'labels' field of model_inputs.
    # The 'labels' field is used during training to calculate the loss and guide model learning.
    model_inputs["labels"] = labels["input_ids"]

    # Returns the prepared inputs and labels as a single dictionary, ready for training.
    return model_inputs

In [12]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [13]:
tokenized_billsum['test'][0]['text']

'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 22513 of the Vehicle Code is amended to read:\n22513.\n(a) (1) It is a misdemeanor for a towing company or the owner or operator of a tow truck to stop or cause a person to stop at the scene of an accident or near a disabled vehicle for the purpose of soliciting an engagement for towing services, either directly or indirectly, to furnish towing services, to move a vehicle from a highway, street, or public property when the vehicle has been left unattended or when there is an injury as the result of an accident, or to accrue charges for services furnished under those circumstances, unless requested to perform that service by a law enforcement officer or public agency pursuant to that agency’s procedures, or unless summoned to the scene or requested to stop by the owner or operator of a disabled vehicle.\n(2) (A) A towing company or the owner or operator of a tow truck summoned, or alleging it was summon

In [14]:
tokenized_billsum['test'][0]['summary']

'Existing law requires a business taking possession of a vehicle from a tow truck to document specified information, including the make, model, and license plate or vehicle identification number of the vehicle. Existing law requires a business taking possession of a vehicle to obtain the specified information from the towing company the next day if the vehicle was dropped off after hours. Existing law requires the information to be maintained for 3 years and to be made available for inspection and copying within 48 hours of a written request by any officer or agent of a police department, a sheriff’s department, the Department of the California Highway Patrol, the Attorney General’s office, the Bureau of Automotive Repair, a district attorney’s office, or a city attorney’s office. A willful violation of these requirements is a misdemeanor, as specified.\nThis bill would specify that a business is required to document the specified information described above when it takes possession of

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [16]:
! pip install -q evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [17]:
import evaluate

rouge = evaluate.load("rouge")

In [18]:
import numpy as np

def compute_metrics(eval_pred):
    # Unpacks the evaluation predictions tuple into predictions and labels.
    predictions, labels = eval_pred

    # Decodes the tokenized predictions back to text, skipping any special tokens (e.g., padding tokens).
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replaces any -100 values in labels with the tokenizer's pad_token_id.
    # This is done because -100 is often used to ignore certain tokens when calculating the loss during training.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decodes the tokenized labels back to text, skipping any special tokens (e.g., padding tokens).
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Computes the ROUGE metric between the decoded predictions and decoded labels.
    # The use_stemmer parameter enables stemming, which reduces words to their root form before comparison.
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Calculates the length of each prediction by counting the non-padding tokens.
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    # Computes the mean length of the predictions and adds it to the result dictionary under the key "gen_len".
    result["gen_len"] = np.mean(prediction_lens)

    # Rounds each value in the result dictionary to 4 decimal places for cleaner output, and returns the result.
    return {k: round(v, 4) for k, v in result.items()}


In [19]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="fine_tuned_t5_small_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

In [22]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.830661,0.129100,0.038100,0.106900,0.107000,19.000000
2,No log,2.617924,0.137100,0.049500,0.115100,0.115100,19.000000
3,No log,2.552830,0.137100,0.050400,0.114600,0.114500,19.000000
4,No log,2.535705,0.138400,0.050800,0.115000,0.115200,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=248, training_loss=3.028234174174647, metrics={'train_runtime': 280.2058, 'train_samples_per_second': 14.118, 'train_steps_per_second': 0.885, 'total_flos': 1070824333246464.0, 'train_loss': 3.028234174174647, 'epoch': 4.0})

In [24]:
trainer.save_model("my_fine_tuned_t5_small_model")


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
#trainer.push_to_hub("Abhi3127/Fine-tuned-t5-small")

In [25]:
text = billsum['test'][100]['text']
text = "summarize: " + text
text

'summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 19780 of the Government Code is amended to read:\n19780.\n(a) Except as provided in Section 19781, a permanent, probationary, or exempt employee who begins active duty within 90 calendar days from the effective date of his or her long-term military leave or within 10 calendar days from the effective date of his or her short-term or emergency military leave and who returns to state service within six months after termination of military service under his or her long-term military leave or within 10 days after termination of military service under his or her short-term or emergency military leave shall be reinstated to his or her former position. Reinstatement to an employee’s former exempt position under this section or Section 19783 shall reestablish the employee’s tenure and civil service reinstatement rights, if any, as they existed immediately prior to his or her military leave pursuant t

In [26]:
from transformers import pipeline

summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model")
pred = summarizer(text)
pred

Token indices sequence length is longer than the specified maximum sequence length for this model (2086 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'a permanent, probationary, or exempt employee who begins active duty within 90 calendar days from the effective date of his or her long-term military leave or within 10 days after termination of military service under his/her short-term or emergency military leave shall be reinstated to the employee’s tenure and civil service reinstatement rights, if any, as they existed immediately prior to the member’s or her military leave pursuant to this section, and who returns to state service within six months after receiving a discharge from military service other than dishonor'}]

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_fine_tuned_t5_small_model")
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs

Token indices sequence length is longer than the specified maximum sequence length for this model (2084 > 512). Running this sequence through the model will result in indexing errors


tensor([[21603,    10,    37,  ...,  1375,     5,     1]])

In [28]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_t5_small_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

In [29]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'SEC. 1. The state of California does enact as follows: SEC. 1. A permanent, probationary, or exempt employee who begins active duty within 90 calendar days from the effective date of his or her long-term military leave or within 10 days after termination of military service under his or her short-term or emergency military leave shall be reinstated to his or her former position. For the purpose of this section any period of rehabilitation afforded by the United States or the'

In [30]:
pred[0]['summary_text']

'a permanent, probationary, or exempt employee who begins active duty within 90 calendar days from the effective date of his or her long-term military leave or within 10 days after termination of military service under his/her short-term or emergency military leave shall be reinstated to the employee’s tenure and civil service reinstatement rights, if any, as they existed immediately prior to the member’s or her military leave pursuant to this section, and who returns to state service within six months after receiving a discharge from military service other than dishonor'

In [31]:
preds = [pred[0]['summary_text']]

In [32]:
labels = [billsum['test'][100]['summary']]

In [33]:
rouge.compute(predictions=preds, references=labels, use_stemmer=True)

{'rouge1': 0.22172949002217296,
 'rouge2': 0.0712694877505568,
 'rougeL': 0.12416851441241684,
 'rougeLsum': 0.17738359201773832}